#### Using Stewart Platform as a tool to understand key robotic concepts
[<img align="right" src="./doc/jupyter_nb/Hexapod_general_Anim.gif" width="20%" height="20%">](./doc/jupyter_nb/Hexapod_general_Anim.gif)

The most basic configuration of Stewart Platforms uses 6 legs, each connecting the base to the platform. They are arranged in 3 pairs, connected with universal joints and actuated linearly. 

We first define the base and platform anchors of the 

In [27]:
import numpy as np
r_B, r_P, alpha_B, alpha_P = 6.2, 5, 0.2269, 0.2269

pi = np.pi
## Define the Geometry of the platform

# Beta (Angle)
# Angle between the plane in which the servo arm moves and the xz-plane of the base CS.
beta = np.array([ 
    pi+pi/2,        
    pi/2,
    2*pi/3+pi+pi/2, 
    2*pi/3+pi/2,
    4*pi/3+pi+pi/2, 
    4*pi/3+pi/2] )

# Theta_B (Polar coordinates)
# Direction of the points where the servo arm is attached to the servo axis.
theta_B = np.array([ 
    alpha_B, 
    alpha_B,
    pi/3 + alpha_B, 
    pi/3 - alpha_B, 
    pi/3 - alpha_B, 
    pi/3 + alpha_B])

# Theta_P (Polar coordinates)
# Direction of the points where the rod is attached to the platform.
theta_P = np.array([ 
    pi/3 - alpha_P, 
    pi/3 - alpha_P, 
    pi/3 + alpha_P, 
    alpha_P,
    alpha_P, 
    pi/3 + alpha_P] )

# Coordinate of the points where servo arms 
# are attached to the corresponding servo axis.
B = r_B * np.array( [ 
    [ np.cos(theta_B[0]), -np.sin(theta_B[0]), 0],
    [ np.cos(theta_B[1]),  np.sin(theta_B[1]), 0],
    [-np.cos(theta_B[2]),  np.sin(theta_B[2]), 0],
    [-np.cos(theta_B[3]),  np.sin(theta_B[3]), 0],
    [-np.cos(theta_B[4]), -np.sin(theta_B[4]), 0],
    [-np.cos(theta_B[5]), -np.sin(theta_B[5]), 0] ])
B = np.transpose(B)
    
# Coordinates of the points where the rods 
# are attached to the platform.
P = r_P * np.array([ 
    [ np.cos(theta_P[0]), -np.sin(theta_P[0]), 0],
    [ np.cos(theta_P[1]),  np.sin(theta_P[1]), 0],
    [ np.cos(theta_P[2]),  np.sin(theta_P[2]), 0],
    [-np.cos(theta_P[3]),  np.sin(theta_P[3]), 0],
    [-np.cos(theta_P[4]), -np.sin(theta_P[4]), 0],
    [ np.cos(theta_P[5]), -np.sin(theta_P[5]), 0] ])
P = np.transpose(P)


In [28]:
import matplotlib.pyplot as plt
import seaborn

def plot_2D_annotate(X, Y):
    X = list(X)
    Y = list(Y)
    ax.plot(X,Y, 'or')  # Plot Points
    # Annotate Points
    for xy in zip(X, Y):                                 
        ax.annotate('(%.5s, %.5s)' % xy, xy=xy, textcoords='data')

    X.append(X[0])
    Y.append(Y[0])
    ax.plot(X,Y)    # Plot lines

seaborn.set(style='ticks')
plt.style.use('dark_background')

fig = plt.figure(figsize=(14, 6), dpi=80)

ax = fig.add_subplot(121)
ax.set_title('Platform Anchors')
plot_2D_annotate(P[0,:], P[1,:])
circle_r = plt.Circle((0, 0), r_P, color='g', fill=False)
ax.add_patch(circle_r)

ax = fig.add_subplot(122)
ax.set_title('Base Anchors')
plot_2D_annotate(B[0,:], B[1,:])
circle_B = plt.Circle((0, 0), r_B, color='g', fill=False)
ax.add_patch(circle_B)

plt.show()

In [49]:
# Definition of the platform home position.
ldl = 9
lhl = 4
z = np.sqrt( ldl**2 + lhl**2 - (P[0] - B[0])**2 - (P[1] - B[1])**2)
home_pos= np.array([0, 0, z[0] ])


# s.home_pos = np.transpose(home_pos)

# Allocate for variables
eg = np.zeros((3,6))
llegl = np.zeros((6))
angles = np.zeros((6))
joint_B = np.zeros((3,6)) 

In [50]:
pi = np.pi

def rotX(phi):
    rotx = np.array([
        [1,     0    ,    0    ],
        [0,  np.cos(phi), np.sin(phi)],
        [0, -np.sin(phi), np.cos(phi)] ])
    return rotx

def rotY(theta):    
    roty = np.array([
        [np.cos(theta), 0, -np.sin(theta) ],
        [0         , 1,     0       ],
        [np.sin(theta), 0,  np.cos(theta) ] ])   
    return roty
    
def rotZ(psi):    
    rotz = np.array([
        [ np.cos(psi), np.sin(psi), 0 ],
        [-np.sin(psi), np.cos(psi), 0 ],
        [   0        ,     0      , 1 ] ])   
    return rotz
    
trans = np.array([2,1,0])
orient = np.array([pi/6, pi/10, pi/8])

trans = np.transpose(trans)
orient = np.transpose(orient)

# Get rotation matrix of platform. RotZ* RotY * RotX -> matmul
R = np.matmul( np.matmul(rotZ(orient[2]), rotY(orient[1])), rotX(orient[0]) )

# Get leg length for each leg
leg = np.repeat(trans[:, np.newaxis], 6, axis=1) + np.repeat(home_pos[:, np.newaxis], 6, axis=1) + np.matmul(np.transpose(R), P) - B 
llegl = np.linalg.norm(leg, axis=0)

Leg = leg + B



In [51]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot3D_line(ax, vec_arr_origin, vec_arr_dest, color_):
    for i in range(6):
        ax.plot([vec_arr_origin[0, i] , vec_arr_dest[0, i]],
        [vec_arr_origin[1, i], vec_arr_dest[1, i]],
        [vec_arr_origin[2, i],vec_arr_dest[2, i]],
        color=color_)

#interactive plotting in separate window
%matplotlib qt 

ax = plt.axes(projection='3d') # Data for a three-dimensional line
ax.set_xlim3d(-10, 10)
ax.set_ylim3d(-10, 10)
ax.set_zlim3d(0, 20)

# ax.add_collection3d(Poly3DCollection([list(np.transpose(s.B))]), zs='z')
ax.add_collection3d(Poly3DCollection([list(np.transpose(B))], facecolors='green', alpha=0.25))

# ax.add_collection3d(base_plot, zs='z')
ax.add_collection3d(Poly3DCollection([list(np.transpose(Leg))], facecolors='blue', alpha=0.25))

plot3D_line(ax, B, Leg, 'yellow')




Since I wanted to build a small hardware prototype, 6 rotational actuators (60g servos)


In [52]:
# ldl = 8
# lhl = 5

# Position of legs, wrt to their individual bases
lx = leg[0, :]
ly = leg[1, :]
lz = leg[2, :]

# Calculate auxiliary quatities g, f and e
g = llegl**2 - ( ldl**2 - lhl**2 )
e = 2 * lhl * lz

# Calculate servo angles for each leg
for k in range(6):
    fk = 2 * lhl * (np.cos(beta[k]) * lx[k] + np.sin(beta[k]) * ly[k])
    
    # The wanted position could be achieved if the solution of this
    # equation is real for all i
    angles[k] = np.arcsin(g[k] / np.sqrt(e[k]**2 + fk**2)) - np.arctan2(fk,e[k])
    
    # Get postion of the point where a spherical joint connects servo arm and rod.
    joint_B[:, k] = np.transpose([ lhl * np.cos(angles[k]) * np.cos(beta[k]) + B[0,k],
                    lhl * np.cos(angles[k]) * np.sin(beta[k]) + B[1,k],
                    lhl * np.sin(angles[k]) ])

#interactive plotting in separate window
%matplotlib qt 

ax = plt.axes(projection='3d') # Data for a three-dimensional line

ax.set_xlim3d(-10, 10)
ax.set_ylim3d(-10, 10)
ax.set_zlim3d(0, 20)

# ax.add_collection3d(Poly3DCollection([list(np.transpose(s.B))]), zs='z')
ax.add_collection3d(Poly3DCollection([list(np.transpose(B))], facecolors='green', alpha=0.25))

# ax.add_collection3d(base_plot, zs='z')
ax.add_collection3d(Poly3DCollection([list(np.transpose(Leg))], facecolors='blue', alpha=0.25))

plot3D_line(ax, B, joint_B, 'red')
plot3D_line(ax, joint_B, Leg, 'black')
plot3D_line(ax, B, Leg, 'yellow')


[<img src="./doc/jupyter_nb/pseudocode.png" width="40%" height="40%">](./doc/jupyter_nb/pseudocode.png)


This is part 1 of 3 in a series
In the next section we discuss how to use this inverse kinematics as a tangible controller, what choices we have an their tradeoffs

Special thanks to Dr. David Navarro for guidance and lecture slides